# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 
import re
pd.set_option('display.max_columns', None)

# 连接数据库

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

#  获取订单数据

In [3]:
# 分期表
sql1 = '''
SELECT
om.create_time 下单日期, om.order_number 订单号, tos.refund_date 预计扣款日期, tos.reality_refund_date 实际扣款日期, tos.money 预计扣款金额, tos.real_pay_money 实际扣款金额, tos.sort 期数
,om.relet_status 是否续租 -- 1 未续租； 2 续租中
FROM db_digua_business.t_postlease_receivables_monitoring tprm 
LEFT JOIN db_digua_business.t_order om on tprm.order_id=om.id
LEFT JOIN db_digua_business.t_merchant tmer on tmer.id=om.merchant_id
LEFT JOIN db_digua_business.t_order_stages tos on tos.order_id=om.id
WHERE  tmer.shop_type != 2 and tmer.`name`!='探路者自营账户' and tmer.`name`!='CPS渠道合作';
'''
# shop_type 店铺类型：1、自营店铺；2、入驻店铺；3、代运营店铺'
df1 = query(sql1)

In [4]:
# 续租表
sql2 = '''
SELECT
om.create_time 下单日期_续租, om.order_number 订单号_续租,  tor.money 预计扣款金额_续租, tor.cycle_day 预计扣款日期_续租, tor.pay_date 实际扣款日期_续租, tor.cycle_status 是否扣款
,om.relet_status 是否续租_续租 -- 1 未续租； 2 续租中
FROM db_digua_business.t_postlease_receivables_monitoring tprm 
LEFT JOIN db_digua_business.t_order om on tprm.order_id=om.id 
LEFT JOIN db_digua_business.t_merchant tmer on tmer.id=om.merchant_id
LEFT JOIN db_digua_business.t_order_relet tor on tor.order_id=om.id
WHERE tmer.shop_type != 2 and tmer.`name`!='探路者自营账户' and tmer.`name`!='CPS渠道合作';
'''
df2 = query(sql2)

In [5]:
# 资方分期表
sql3 = '''
SELECT 
om.create_time 下单日期_资方, om.order_number 订单号_资方, ymos.money 预计扣款金额_资方, ymos.part_payment 实际扣款金额_资方, ymos.refund_date 预计扣款日期_资方, ymos.reality_refund_date 实际扣款日期_资方, ymos.sort 期数_资方
,om.relet_status 是否续租_资方 -- 1 未续租； 2 续租中
FROM db_digua_business.t_postlease_receivables_monitoring tprm 
LEFT JOIN db_digua_business.t_order om on tprm.order_id=om.id
LEFT JOIN db_digua_business.t_merchant tmer on tmer.id=om.merchant_id
LEFT JOIN db_rent.ya_merchant_order_stages ymos on ymos.order_id=om.id
WHERE  tmer.shop_type != 2 and tmer.`name`!='探路者自营账户' and tmer.`name`!='CPS渠道合作';
'''
df3 = query(sql3)

In [6]:
df1.head(24)

,下单日期,订单号,预计扣款日期,实际扣款日期,预计扣款金额,实际扣款金额,期数,是否续租
0,2022-07-05 08:51:37,A202207050851373,2022-07-05,2022-07-05 10:41:10,357.95,357.95,1,1
1,2022-07-05 08:51:37,A202207050851373,2022-08-05,2022-08-05 02:00:03,357.95,357.95,2,1
2,2022-07-05 08:51:37,A202207050851373,2022-09-05,2022-09-05 02:00:03,357.95,357.95,3,1
3,2022-07-05 08:51:37,A202207050851373,2022-10-05,2022-10-09 02:00:03,357.95,357.95,4,1
4,2022-07-05 08:51:37,A202207050851373,2022-11-05,2022-11-08 02:00:03,357.95,357.95,5,1
5,2022-07-05 08:51:37,A202207050851373,2022-12-05,2022-12-08 02:00:06,357.95,357.95,6,1
6,2022-07-05 08:51:37,A202207050851373,2023-01-05,2023-01-07 02:00:03,357.95,357.95,7,1
7,2022-07-05 08:51:37,A202207050851373,2023-02-05,2023-02-06 02:00:02,357.95,357.95,8,1
8,2022-07-05 08:51:37,A202207050851373,2023-03-05,2023-03-07 02:00:03,357.95,357.95,9,1
9,2022-07-05 08:51:37,A202207050851373,2023-04-05,2023-04-01 09:00:03,357.95,357.95,10,1


In [7]:

df2['实际扣款日期_续租'] = df2['实际扣款日期_续租'].dt.strftime('%Y-%m-%d')
df2.head(24)

,下单日期_续租,订单号_续租,预计扣款金额_续租,预计扣款日期_续租,实际扣款日期_续租,是否扣款,是否续租_续租
0,2022-07-05 08:51:37,A202207050851373,354.0,None,2023-07-07,NaN,1
1,2022-07-05 08:51:37,A202207050851373,354.0,2023-09-01,2023-08-07,1.0,1
2,2022-07-05 08:51:37,A202207050851373,354.0,2023-11-05,2023-09-06,1.0,1
3,2022-07-05 08:51:37,A202207050851373,365.8,None,2023-09-29,NaN,1
4,2022-07-05 08:51:37,A202207050851373,354.0,2023-11-02,2023-11-02,2.0,1
5,2022-07-05 08:51:37,A202207050851373,354.0,2023-12-03,2023-12-03,2.0,1
6,2022-07-05 08:51:37,A202207050851373,354.0,2024-01-05,2023-12-18,2.0,1
7,2022-07-05 08:51:37,A202207050851373,354.0,2024-02-05,2024-01-22,2.0,1
8,2022-07-05 08:51:37,A202207050851373,354.0,2024-03-05,2024-02-28,2.0,1
9,2022-07-05 08:51:37,A202207050851373,354.0,2024-04-05,2024-04-03,2.0,1


In [8]:
df3['实际扣款日期_资方']=df3['实际扣款日期_资方'].dt.strftime('%Y-%m-%d')
df3.head(24)

,下单日期_资方,订单号_资方,预计扣款金额_资方,实际扣款金额_资方,预计扣款日期_资方,实际扣款日期_资方,期数_资方,是否续租_资方
0,2022-07-05 08:51:37,A202207050851373,357.95,357.95,2022-07-05,2022-07-05,1.0,1
1,2022-07-05 08:51:37,A202207050851373,357.95,357.95,2022-08-05,2022-08-05,2.0,1
2,2022-07-05 08:51:37,A202207050851373,357.95,357.95,2022-09-05,2022-09-05,3.0,1
3,2022-07-05 08:51:37,A202207050851373,357.95,357.95,2022-10-05,2022-10-09,4.0,1
4,2022-07-05 08:51:37,A202207050851373,357.95,357.95,2022-11-05,2022-11-08,5.0,1
5,2022-07-05 08:51:37,A202207050851373,357.95,357.95,2022-12-05,2022-12-08,6.0,1
6,2022-07-05 08:51:37,A202207050851373,357.95,357.95,2023-01-05,2023-01-07,7.0,1
7,2022-07-05 08:51:37,A202207050851373,357.95,357.95,2023-02-05,2023-02-06,8.0,1
8,2022-07-05 08:51:37,A202207050851373,357.95,357.95,2023-03-05,2023-03-07,9.0,1
9,2022-07-05 08:51:37,A202207050851373,357.95,357.95,2023-04-05,2023-04-01,10.0,1


# 分期表

In [9]:
df_13 = pd.merge(df1, df3, left_on=['订单号','期数'], right_on=['订单号_资方','期数_资方'], how='left').drop(columns=['下单日期_资方','订单号_资方'])


In [10]:
df_13_1 = df_13[(df_13['实际扣款日期_资方'].notnull())&(df_13['实际扣款金额_资方']==0)]
# df_13_1[df_13_1['实际扣款金额_资方']==0]
df_13_1['实际扣款日期_资方'].drop_duplicates()

90       2023-03-18
1362     2023-03-19
9001     2023-03-20
10402    2023-04-17
12360    2023-03-22
Name: 实际扣款日期_资方, dtype: object

In [11]:
df_13_1[pd.to_datetime(df_13_1['实际扣款日期_资方'])=='2023-03-22'][['订单号','实际扣款金额_资方','实际扣款日期_资方']].drop_duplicates()
# 除了2023-03-18，04-17，03-20其他日期的用户都是支付状态是已完成

,订单号,实际扣款金额_资方,实际扣款日期_资方
12360,A2023032216313863,0.0,2023-03-22


In [12]:
df_13_2 = df_13[(df_13['实际扣款日期_资方'].isnull())&(df_13['实际扣款金额_资方']!=0)]
# df_13_2[['订单号','实际扣款金额_资方','预计扣款日期_资方','实际扣款日期_资方']].drop_duplicates()
# 待归还（归还逾期），已买断的买断价会分成每期的扣款金额，且会有实扣日期，当尾数不够扣款金额时，就不会有实扣日期

In [13]:
# df1[df1['订单号']=='A20231103145335272']

In [14]:
# df3[df3['订单号_资方']=='A20231103145335272']

In [15]:
# (df2[df2['是否续租']==2].groupby('订单号_续租').agg({'订单号_续租':'count'})+12).shape
# df3[df3['是否续租']==2]['订单号_资方'].drop_duplicates()

In [19]:
df_133 = (df2[df2['是否续租_续租']==2].groupby('订单号_续租').agg({'订单号_续租':'count'})+12)
df_133.rename(columns={'订单号_续租': '期数_all'}, inplace=True)
df_13133 = pd.merge(df_13, df_133, left_on='订单号', right_on='订单号_续租', how='left')
df_133
# df2[df2['是否续租']==2].groupby('订单号_续租')['订单号_续租'].count()+12 (294478, 13) (29532, 14)

,期数_all
订单号_续租,
A2022072701455912,26
A202207282047042,25
A202208031654408,29
A202208061505082,32
A202208091601222,24
A2022081622545014,25
A202208221401171,26
A202209070712512,36
A202209071224329,25


# 续租表

In [17]:
df_23 = pd.merge(df2, df3, left_on=['订单号_续租', '实际扣款日期_续租'], right_on=['订单号_资方', '实际扣款日期_资方'], how='left').drop(columns=['下单日期_资方','订单号_资方'])
# df_23[(df_23['实际扣款日期_资方'].notnull())&(df_23['是否扣款']==1)]
# 资方订单分期表数据与运营后台一致，续租表数据与运营后台不一致，表现为后台显示已扣款而续租表显示没有扣款

In [18]:
pd.set_option('display.max_rows', None)
df_df = df_23[(df_23['实际扣款日期_资方'].isna())&(df_23['是否扣款']==2)]['订单号_续租'].drop_duplicates()
# df_df[df_df.str.contains('A2023')]
# 2022年的数据，续租表和资方分期表的数据不一致，且与后台的数据也不一致
# 2023年的数据，续租表会有续租的数据且有扣款记录，但资方分期表和运营后台都没有续租数据